In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import time
from joblib import Parallel, delayed
import datetime
import os

### First we will define a function to set the seed for reproducability and set the device

In [2]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # ensures deterministic algorithms
    torch.backends.cudnn.benchmark = False     # can slow down, but more reproducible

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Next we can define the RNN class and TimeSeries datasets from [Project 3](https://github.com/kennethwirjadisastra/Stat155/tree/main/Project3).

In [3]:
class WeatherRNN(nn.Module):
  def __init__(self, input_size:int, hidden_size:int, output_size:int, num_layers:int=1, rnn_type:str='RNN', dropout:float=0.0):
    super().__init__()

    # potential RNN classes
    rnn_options = {"RNN": nn.RNN,
                   "LSTM": nn.LSTM,
                   "GRU": nn.GRU}

    if rnn_type not in rnn_options:
      raise ValueError(f'rnn_type must be one of {list(rnn_options.keys())}')

    # force dropout to be 0 if num_layers == 1
    dropout = dropout if num_layers > 1 else 0.0

    self.rnn_type = rnn_type

    # define the rnn
    self.rnn = rnn_options[rnn_type](input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    # fully connected linear layers
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x, h0=None):
    batch_size = x.size(0)
    out, _ = self.rnn(x) if h0 is None else self.rnn(x, h0)
    out = self.fc(out[:,-1,:])
    return out, _
  

class TimeSeries(Dataset):
  def __init__(self, data, seq_len):
    self.data = torch.tensor(data, dtype=torch.float32)
    self.seq_len = seq_len

  def __len__(self):
    # subtract self.seq_len to account for data where previous sequence is unknown
    return len(self.data) - self.seq_len

  def __getitem__(self, start_idx):
    end_idx = start_idx + self.seq_len

    # generate the sequence of size seq_len and the target that follows directly after
    X_seq = self.data[start_idx:end_idx].unsqueeze(-1) # (seq_len x 1)
    y = self.data[end_idx].unsqueeze(-1) # (1,)

    return X_seq, y

### We will also reuse the same functions defined in [Project 3](https://github.com/kennethwirjadisastra/Stat155/tree/main/Project3).

In [4]:
def read_data(stations):
    # create array to store each dataframe created
    df_dict = {}         # each value in df_dict will be an dictionary with values of pd.DataFrame objects
                         # the key of each dataframe determines the type of dataset as shown below

    dataset_types = {
        0: "train",      # first element
        1: "validation", # second element
        2: "test"        # third element
    }

    # recover necessary data from each csv file
    for station in stations:

        for j in dataset_types.keys():
            df = pd.read_csv(f'../Project1/{dataset_types[j]}_data/{station}_weather_{dataset_types[j]}.csv')

            df["DATE"] = pd.to_datetime(df["DATE"])
        
            df.set_index("DATE", inplace=True)
        
            df = pd.DataFrame(df["TMAX"])

            # add dataframe to array
            df_dict[station] = [df] if station not in df_dict.keys() else df_dict[station] + [df]

    return df_dict

In [5]:
def normalize(df_arr, transformation):
    # array to store normalized data
    norm_df_arr = []

    # normalize data and append to array
    for df in df_arr:
        df["NORM"] = transformation(df["TMAX"].values.reshape(-1,1))
        norm_df_arr.append(df)
    
    return norm_df_arr

def load_data(data_norm, seq_len, batch_size=32, shuffle=False, drop_last=True):
    dataset = TimeSeries(data_norm.values, seq_len=seq_len)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=True)

    return dataset, dataloader

In [6]:
def train_RNN(model, train_dataloader, validation_dataloader, num_epochs=50, 
              criterion=nn.MSELoss(), optimizer=optim.Adam, eta=1.e-3, device=device):
    
    model = model.to(device)

    training_loss = np.empty(num_epochs) * np.nan
    validation_loss = np.empty(num_epochs) * np.nan

    optimizer = optimizer(model.parameters(), lr=eta)

    #print("Training ...")

    #start_time = time.time()

    for epoch in range(num_epochs):
        # training and validation loss for each epoch
        train_epoch_loss = 0
        validation_epoch_loss = 0

        # reset hidden state after each epoch
        hidden = None

        model.train()
        for x_batch, y_batch in train_dataloader:
            if hidden is not None:
                if model.rnn_type == "LSTM":
                    hidden = (hidden[0].detach(), hidden[1].detach())
                else:
                    hidden = hidden.detach()

            out, hidden = model(x_batch, hidden)
            loss = criterion(out, y_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_epoch_loss += loss.item()
        
        # average loss over each batch
        train_epoch_loss /= len(train_dataloader)

        # save losses for plotting later
        training_loss[epoch] = train_epoch_loss

        
        model.eval()
        # reset hidden state
        hidden = None
        with torch.no_grad():
            for x_batch, y_batch in validation_dataloader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                if hidden is not None:
                    if model.rnn_type == "LSTM":
                        hiden = (hidden[0].detach(), hidden[1].detach())
                    else:
                        hidden = hidden.detach()

                out, hidden = model(x_batch, hidden)
                loss = criterion(out, y_batch)
                validation_epoch_loss += loss.item()
            
            validation_epoch_loss /= len(validation_dataloader)
        
        validation_loss[epoch] = validation_epoch_loss

        if epoch % 10 == 9:
            print(f'Epoch {epoch+1}: \n\tTraining Loss = {train_epoch_loss}\n\tValidation Loss = {validation_epoch_loss}')

    #end_time = time.time()

    #print("Done!")
    #print(f'Total time (seconds): {end_time-start_time}')

    return training_loss, validation_loss

### Now we can load the training, validation, and testing datasets for each of the four stations into a dictionary named `datasets`.

In [7]:
stations = ["APA", "central_park", "DEN", "water_dept"]
dataframes = read_data(stations)

In [8]:
scaler = MinMaxScaler()

for station in stations:
    dataframes[station] = normalize(dataframes[station], scaler.fit_transform)

In [9]:
seq_len = 7

dataset_types = {
        0: "train",      # first element
        1: "validation", # second element
        2: "test"        # third element
    }

loaders = {}

for station, (train_data, validation_data, test_data) in dataframes.items():
    for i in range(len(dataset_types)):
        dataset_type = dataset_types[i]

        df = dataframes[station][i]["NORM"]

        _, dataloader = load_data(df, seq_len)

        if station not in loaders.keys():
            loaders[station] = {dataset_type: dataloader}
        else:
            loaders[station][dataset_type] = dataloader

In [10]:
current_date = datetime.date.today()

year = current_date.year
month = current_date.month
date = current_date.day


os.makedirs(f'{year}_{month}_{date}_model_save', exist_ok=True)

In [11]:
def train_member(model, train_loader, val_loader, num_epochs):
    train_loss, val_loss = train_RNN(model, train_loader, val_loader, num_epochs=num_epochs)
    return train_loss, val_loss


model_types = ["RNN", "LSTM", "GRU"]
hidden_sizes = [16, 32, 64]
stations = ["APA", "central_park", "DEN", "water_dept"]

num_epochs = 30
ensemble_size = 10
loss = np.zeros((len(model_types), 
                 len(hidden_sizes), 
                 len(stations), 
                 ensemble_size, 
                 num_epochs,
                 2)) # training and validation loss for each configuration

all_models = {} #store all model ensembles in a dictionary

num_models = len(model_types) * len(hidden_sizes) * len(stations) * ensemble_size

start_time = time.time()

model_id = 1

for m, model_type in enumerate(model_types):
    for h, hidden_size in enumerate(hidden_sizes):
        for s, station in enumerate(stations):
            key = f'{model_type}_{hidden_size}_{station}'
            train_loader, validation_loader, test_loader = loaders[station].values()

            ensemble = [WeatherRNN(input_size=1,
                                   hidden_size=hidden_size,
                                   output_size=1,
                                   num_layers=1,
                                   rnn_type=model_type) for _ in range(ensemble_size)]

            # Parallel training
            results = Parallel(n_jobs=-1)(
                delayed(train_member)(model, train_loader, validation_loader, num_epochs)
                for model in ensemble
            )

            for e, (train_loss, val_loss) in enumerate(results):
                loss[m, h, s, e, :, :] = np.stack((train_loss, val_loss), axis=1)
            
            all_models[key] = ensemble # add model ensemble to dictionary

            print(f'{10*model_id}/{num_models} models complete')
            model_id += 1

                
end_time = time.time()
print(f'Time Elapsed: {end_time-start_time:.3f}')

10/360 models complete
20/360 models complete
30/360 models complete
40/360 models complete
50/360 models complete
60/360 models complete
70/360 models complete
80/360 models complete
90/360 models complete
100/360 models complete
110/360 models complete
120/360 models complete
130/360 models complete
140/360 models complete
150/360 models complete
160/360 models complete
170/360 models complete
180/360 models complete
190/360 models complete
200/360 models complete
210/360 models complete
220/360 models complete
230/360 models complete
240/360 models complete
250/360 models complete
260/360 models complete
270/360 models complete
280/360 models complete
290/360 models complete
300/360 models complete
310/360 models complete
320/360 models complete
330/360 models complete
340/360 models complete
350/360 models complete
360/360 models complete
Time Elapsed: 1139.811


### Save models

In [17]:
for key, ensemble in all_models.items():
    model_type, hidden_size, station = key.rsplit("_", maxsplit=2)
    for i, model in enumerate(ensemble):
        path = f"{year}_{month}_{date}_model_save/{model_type}_{hidden_size}_{station}_model{i}.pt"
        torch.save(model.state_dict(), path)